<a href="https://colab.research.google.com/github/Milan-Johnson/AudioVideoSummarization/blob/master/AudioVideoSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Choose Model for Language Summarization(Audio)
# @markdown BART : For better accuracy, but lenghty (FAST) \
# @markdown T-5  : Summary may be concise, but tends to Hallucinate (SLOW)
# hdjdjd
Model = "BART" # @param ["BART", "T5"]
print(Model)

BART


In [ ]:
from google.colab import drive # loads a library to mount your google drive
drive.mount('/content/drive')
#!rm /content/drive/MyDrive/AVS/*
#!rm -r /content/drive/MyDrive/AVS/Frames/*
!mkdir /content/drive/MyDrive/AVS
!mkdir /content/drive/MyDrive/AVS/preprocessed_img
!mkdir /content/drive/MyDrive/AVS/summarized_frames
!mkdir /content/drive/MyDrive/AVS/Frames
!mkdir /content/drive/MyDrive/AVS/DATA

In [ ]:
#!pip install numpy
#!pip install tensorflow
#!pip install keras
#!pip install scikit-learn
#!pip install matplotlib
#!pip install scipy
#!pip install tkinter
#!pip install tqdm
#!pip install seaborn
#!pip install opencv-python
#!pip install librosa
#!pip install ffmpeg-python
!pip install mutagen
!pip install git+https://github.com/openai/whisper.git  # audio to text
!sudo apt update && sudo apt install ffmpeg
!pip install transformers                               # sumarization
!pip install gTTS

In [ ]:
import os
import sys
import math
import imageio
import cv2
import random
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from imageio import imread, imsave
from os.path import isfile, join
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import DenseNet121
from tkinter import filedialog
from os import listdir
from mutagen.mp3 import MP3
from PIL import Image
from pathlib import Path
from gtts import gTTS
from IPython.display import Audio
from moviepy import editor
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
def FRAME_EXT(video_path, output_dir, interval=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not find path'")
        return
    os.makedirs(output_dir, exist_ok=True)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % interval == 0:
            frame_path = os.path.join(output_dir, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_path, frame)
        frame_count += 1
    cap.release()

In [ ]:
input_directory = "/content/drive/My Drive/AVS/DATA/"
Frame_directory = "/content/drive/My Drive/AVS/Frames/"

# Get the list of files in the input directory and sort them by modification time
videofiles = sorted(os.listdir(input_directory), key=lambda x: os.path.getmtime(os.path.join(input_directory, x)))
i = 0
for filename in videofiles:
    if filename.endswith('.mp4'):
       video_path = os.path.join(input_directory, filename)
       FRAME_EXT(video_path, Frame_directory +str(filename), interval=30)
       file_path = filename
    i+=1
print("Frame extraction is completed")
 #preprocessing using bilateral filter
 #obtaining input image


Frame extraction is completed


In [ ]:
print(file_path,filename,input_directory+file_path)
file_source = input_directory+file_path

7.mp4 7.mp4 /content/drive/My Drive/AVS/DATA/7.mp4


In [ ]:
preprocessimg_path="/content/drive/My Drive/AVS/preprocessed_img/"

i = 0
for filename1 in file_path:
    from pathlib import Path
    gg=Path(file_path).resolve().stem
    video_path1 = os.path.join(Frame_directory, gg+str('.mp4'))
    for filename in  listdir(video_path1):
        video_path = os.path.join(video_path1,str(filename))
        img = cv2.imread(video_path)
        img = cv2.resize(img,[224,224])
        # bilateral filter
        filtered_img1  = cv2.bilateralFilter(img,9,75,75)

        prefn = os.path.join(preprocessimg_path, filename)
        cv2.imwrite(prefn, filtered_img1 )
        mset1 = np.square(np.subtract(img,filtered_img1)).mean()
        MSR1=np.mean(filtered_img1)/np.std(filtered_img1)
        PSNRVAL2=10.0*math.log10((255*255)/mset1)
        print('****** Bilateral filter results ****** ')
        print(MSR1)
        print(PSNRVAL2)
    i+=1
print("Pre processing is completed")

In [ ]:
# Feature extraction using DenseNet121 as the base model
def load_data(dir_list, image_size):


    # load all images in a directory
    data = []
    labl = []
    image_width, image_height = image_size

    base_model = DenseNet121(include_top=False,weights=None,classes = 2,input_tensor=None, input_shape=(224,224,3))
    for directory in dir_list:
        for filename in listdir(directory):
            print(filename)
            # load the image
            path=directory  + filename
            print(path)
            image  = cv2.imread(path,3)
            imgk = np.expand_dims(image, axis=0)
            print(np.shape(image))
            resincepfeat_par = base_model.predict(imgk)
            print(np.shape(resincepfeat_par))
            datak= np.array(resincepfeat_par)
            datak= (np.mean(datak,1))
            datak= (np.mean(datak,2))
            datak1= (np.mean(datak))

            if datak1 < 2:
                labl.append([0])
                data.append(datak)

            elif datak1 > 2:
                labl.append([1])
                data.append(datak)

    return data, labl,directory

IMG_WIDTH, IMG_HEIGHT = (224,224)

In [ ]:
preprocessimg_path="/content/drive/My Drive/AVS/preprocessed_img/"
print(listdir(preprocessimg_path))
#model = tf.keras.models.load_model('model.h5')
data, labl,directory = load_data([preprocessimg_path], (IMG_WIDTH, IMG_HEIGHT))
labl = np.array(labl)
data = np.array(data)
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
AA=np.shape(data)
model = Sequential()
### First GRU layer with Dropout regularisation(Input)
model.add(GRU(units=50, return_sequences=True, input_shape=(1, 7), activation='tanh'))
model.add(Dropout(0.2))
### Second GRU layer(Hidden)
model.add(GRU(units=50, return_sequences=True, input_shape=(1, 7), activation='tanh'))
model.add(Dropout(0.2))
### Third GRU layer(Reset)
model.add(GRU(units=50, return_sequences=True, input_shape=(1, 7), activation='tanh'))
model.add(Dropout(0.2))
### Fourth GRU layer(Update)
model.add(GRU(units=50, activation='tanh'))
model.add(Dropout(0.2))
### The output layer
model.add(Dense(units=1))
import tensorflow as tf
epochs = 50
learning_rate = 0.01
decay_rate = learning_rate / epochs
optimizer = tf.keras.optimizers.legacy.Adam(lr=0.001,decay=decay_rate)
#model.compile(optimizer=SGD(lr=0.001, decay=1e-7, momentum=0.9, nesterov=False),loss='mean_squared_error')
model.compile(optimizer=optimizer,loss='mean_squared_error')
### Fitting to the training set
model.fit(data,labl,epochs=50,batch_size=150)
out = model.predict(data)
### The LSTM architecture
model1 = Sequential()
### First LSTM layer with Dropout regularisation
model1.add(LSTM(units=50, return_sequences=True, input_shape=(1,7)))
model1.add(Dropout(0.2))
### Second LSTM layer
model1.add(LSTM(units=50, return_sequences=True))
model1.add(Dropout(0.2))
### Third LSTM layer
model1.add(LSTM(units=50, return_sequences=True))
model1.add(Dropout(0.2))
### Fourth LSTM layer
model1.add(LSTM(units=50))
model1.add(Dropout(0.2))
### The output layer
model1.add(Dense(units=1))
model1.compile(optimizer='rmsprop',loss='mean_squared_error')
### Fitting to the training set
model1.fit(data,labl,epochs=50,batch_size=32)
out1 = model1.predict(data)

out=np.round(out*1000)
out1=np.round(out1*1000)
aa=np.shape(out)
flab=np.zeros([aa[0],1], dtype=int)
##
for ip in range(0,aa[0]):
    if [out[ip]]==out1[ip]:
        flab[ip]=out[ip]
    else:
        flab[ip]=out1[ip]
##
list=np.unique(flab, return_index=True)
index=list[1]
index=np.sort(index)
print(index)
#print("Accuracy:",metrics.accuracy_score(c1, c))

In [ ]:
print(list)

(array([ 395,  423,  451,  506,  519,  573,  596,  642,  693,  705,  708,
        721,  756,  766,  773,  777,  788,  789,  813,  818,  820,  822,
        830,  840,  849,  862,  865,  868,  870,  872,  877,  878,  881,
        882,  883,  885,  886,  890,  893,  898,  899,  900,  901,  902,
        906,  907,  908,  909,  910,  911,  919,  925,  938,  939,  940,
        942,  943,  944,  946,  947,  950,  953,  956,  958,  960,  961,
        963,  965,  966,  967,  969,  970,  977,  978,  979,  984,  985,
        986,  987,  988,  993,  994,  996,  997,  998, 1006, 1007, 1008,
       1009, 1010, 1011, 1013, 1014, 1015, 1016, 1022, 1023, 1026, 1028,
       1029, 1030, 1031, 1032, 1033, 1034, 1036, 1037, 1038, 1048, 1050,
       1051, 1065, 1077, 1082, 1083, 1084, 1087]), array([ 86,  50,  90,  25,  24,  89,  27,  63,  51,  26,  62, 142, 117,
       118, 129, 119,  29, 134,  20,  22,  28,   6,  23,   5,   0,  46,
        71,  73,  72,  69,  12, 111, 110,  94,  93,  92, 109,   9,   7,
  

In [ ]:
# summarized frames

preprocessimg_path="/content/drive/My Drive/AVS/preprocessed_img/"
summarizedimg_path="/content/drive/My Drive/AVS/summarized_frames/"
fn=29
for filename in  index:

  #print(fn)
  video_path = preprocessimg_path+'frame_'+str(filename+filename*fn)+'.jpg'
  #fn=fn+29
  print(filename+fn)
  print(video_path)
  img = cv2.imread(video_path)
  prefn = summarizedimg_path+str(filename)+'.jpg'
  cv2.imwrite(prefn, img )
print("Video Summarization is completed")

In [ ]:
!pip install git+https://github.com/openai/whisper.git  # audio to text
!sudo apt update && sudo apt install ffmpeg
!pip install transformers                               # sumarization
!pip install gTTS

In [ ]:
!echo $file_source

/content/drive/My Drive/AVS/DATA/7.mp4


In [ ]:
!cp "$file_source" videoIn.mp4

In [ ]:
!whisper videoIn.mp4 --model medium --language English

In [ ]:
input_directory = "/content/videoIn.txt"
#input_directory = "/content/7.txt"
file = open(input_directory,'r')
my_text = file.read()
print(my_text)
file.close()
wordCount = len(my_text.split())
sumMax = wordCount - wordCount//10
sumMin = wordCount//2
print(wordCount,sumMax,sumMin)

In [ ]:
if Model == "BART":
  summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
  summary = summarizer(my_text, max_length=sumMax, min_length=sumMin, do_sample=False)
  summaryText = summary[0]['summary_text']
  print("BART : ", summary[0]['summary_text'])
else:
  my_text = "summarise : " + my_text
  model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
  tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
  inputs = tokenizer(my_text, return_tensors="pt")
  outputs = model.generate(**inputs, \
                        min_length=20, \
                        max_new_tokens=512, \
                        length_penalty = 2, \
                        num_beams=16, \
                        temperature=0.9, \
                        no_repeat_ngram_size=2, \
                        #num_return_sequences 2,\
                        early_stopping=True)
  output_text_Flan_t5 = tokenizer.batch_decode(outputs, \
                                               skip_special_tokens=True)
  summaryText = str(output_text_Flan_t5)
  print ("T5 : ",summaryText)

In [ ]:
sumCount = len(summaryText.split())
print(sumCount)

201


In [ ]:
file = open("summary.txt",'w')
#file.writelines(summary[0]['summary_text'])
file.writelines(summaryText)
file.close()

In [ ]:
#tts = gTTS(summary[0]['summary_text'])
tts = gTTS(summaryText)
sound_file = "/content/drive/My Drive/out.mp3"
tts.save(sound_file)

Audio(sound_file, autoplay=False)

In [ ]:
summarizedimg_path = "/content/drive/MyDrive/AVS/summarized_frames"

In [ ]:
# fusion


video_path = input_directory
images_path = summarizedimg_path
audio_path = "/content/drive/My Drive/out.mp3"
audio = MP3(audio_path)
audio_length = audio.info.length
list_of_images = []
for image_file in os.listdir(images_path):
    if image_file.endswith('.png') or image_file.endswith('.jpg'):
        image_path = os.path.join(images_path, image_file)
        image = Image.open(image_path).resize((400, 400), Image.ANTIALIAS)
        list_of_images.append(image)
print(audio_length)
duration = audio_length/len(list_of_images)
print(duration)
imageio.mimsave('images.gif', list_of_images, fps=1/duration)

video = editor.VideoFileClip("images.gif")
audio = editor.AudioFileClip(audio_path)
final_video = video.set_audio(audio)
os.chdir("/content/drive/My Drive/AVS/")
final_video.write_videofile(fps=1/duration, codec="libx264", filename="videok.mp4")


In [ ]:
#@title -
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/drive/My Drive/AVS/videok.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=650 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#!rm -r /content/drive/MyDrive/AVS/preprocessed_img/*
#!rm -r /content/drive/MyDrive/AVS/summarized_frames/*
#!rm -r /content/drive/MyDrive/AVS/Frames/*
#!rm -r /content/drive/MyDrive/AVS/DATA/*

In [ ]:
#!rm /content/drive/MyDrive/AVS/*
#!rm -r /content/drive/MyDrive/AVS/Frames/*